In [1]:
#imports
import os
import traceback
from owlready2 import get_ontology
from owlready2 import sync_reasoner_hermit
from nltk.wsd import lesk
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from rdflib import Graph

In [3]:
#load ontology and thesaurus
directoryOntology = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
filenameOntology = 'oec-extracted.owl'
file_pathOntology = os.path.join(directoryOntology, filenameOntology)
try:
    # Loading the ontology
    onto = get_ontology("file://" + file_pathOntology).load()
    print("Ontology loaded successfully!")

except Exception as e:
    # if load fails
    print("Error occurred during ontology loading:")
    print(e)

Ontology loaded successfully!


In [ ]:
#add termContentString based on lexicon, context and wordnet
def extract_context_meaning(term_individual, nltk_context):
    context_meaning = ""
    connected_to = []

    try:
        if hasattr(term_individual, "termHasMeaningByContext"):
            related_property = term_individual.termHasMeaningByContext
            for related_ind in related_property:
                related_ind_name = related_ind.name if related_ind.name else ""
                connected_to.append(related_ind_name)

        if hasattr(term_individual, "termLexicon") and term_individual.termLexicon:
            term_lexicon = term_individual.termLexicon[0]  # Assuming a single value
            if term_lexicon in nltk_context.get("ContextDERHousehold", {}):
                context_meaning = nltk_context["ContextDERHousehold"][term_lexicon]
                source = "NLTK Context"
            else:
                synsets = wordnet.synsets(term_lexicon)
                if synsets:
                    context_meaning = synsets[0].definition()
                    source = "WordNet"

            if context_meaning:
                if not hasattr(term_individual, "termContentString"):
                    term_individual.termContentString = []
                term_individual.termContentString.append(context_meaning)
                return context_meaning, connected_to, term_lexicon, source
            else:
                return None, connected_to, term_lexicon, source
        else:
            return None, connected_to, None, None
    except AttributeError:
        return None, [], None, None

def process_individuals(onto, file_path):
    matching_individuals = [ind for ind in onto.individuals() if ind.name.startswith("ExchangedTerm")]

    for term_individual in matching_individuals:
        context_meaning, connected_to, term_lexicon, source = extract_context_meaning(term_individual, nltk_context)

        if context_meaning:
            with onto:
                # Syncs and ave changes to the ontology
                sync_reasoner()  
                #path via IRI does not work for this
                onto.save(file_path)
                
            print(f"Individual: '{term_individual.name}'")
            print(f" - Object Property: 'termHasMeaningByContext'")
            print(f" - Connects to: {', '.join(connected_to) if connected_to else 'None'}")
            if term_lexicon:
                print(f" - Term Lexicon: {term_lexicon}")
            print(f" - Context Meaning: {context_meaning} (Source: {source})")
        else:
            print(" - No matching context found. NLTK WordNet used.")

def main():
    directory = 'C:\\Users\\Matheus\\Documents\\GitHub\\Papers\\Enabling Intelligent Data Exchange in the Brazilian Energy Sector A Context-Aware Ontological Approach\\ontology\\imports'
    filename = 'oec-extracted.owl'
    file_path = os.path.join(directory, filename)

    try:
        onto = get_ontology("file://" + file_path).load()
        print("Ontology loaded successfully!")
        process_individuals(onto, file_path)
    except Exception as e:
        print("Error occurred during ontology loading:")
        print(e)

if __name__ == "__main__":
    nltk_context = {
        "ContextDERHousehold": {
            "Storage": "cells",
            "Solar Energy": "energy produced via sunrays, using solar panels"
        }
    }
    main()
